NLP中预训练模型的灵感来自于图像领域的预训练模型。在CV领域为什么会出现预训练模型？
* 当训练数据少，很难很好地训练复杂的网络时，可以将在复杂的模型上预训练好的参数进行复用。这样原先训练不了的任务就能解决了，即使手头任务训练数据也不少，加个预训练过程也能极大加快任务训练的收敛速度；
* 预训练好的网络参数，尤其是底层的网络参数抽取出特征跟具体任务越无关，越具备任务的通用性，所以这是为何一般用底层预训练好的参数初始化新任务网络参数的原因。而高层特征跟任务关联较大，实际可以不用使用，或者采用Fine-tuning用新数据集合清洗掉高层无关的特征抽取器。

$\color{#FF0000}{预训练的本质：}$ 

通过设计好一个网络结构来**做语言模型任务**，然后把大量甚至是无穷尽的无标注的自然语言文本利用起来，预训练任务把大量**语言学**知识抽取出来编码到网络结构中，当手头任务带有标注信息的数据有限时，这些先验的语言学特征对手头任务有极大的特征补充作用，因为当数据有限的时候，很多语言学现象是覆盖不到的，泛化能力就弱，集成尽量通用的语言学知识自然会加强模型的泛化能力。如何引入先验的语言学知识其实一直是NLP尤其是深度学习场景下的NLP主要目标之一，不过一直没有太好的解决办法，而ELMO/GPT/Bert的这种两阶段模式看起来无疑是解决这个问题自然又简洁的方法，这也是这些方法的主要价值所在。

### 神经网络语言模型（NNLM）

在NLP中做预训练一般选择用语言模型任务来做。什么是语言模型？其实看下图就明白了，为了能够量化地衡量哪个句子更像一句人话，可以设计一个函数，核心思想是根据句子里面前面的一系列前导单词预测后面跟哪个单词的概率大小（理论上除了上文之外，也可以引入单词的下文联合起来预测单词出现概率）。句子里面每个单词都有个根据上文预测自己的过程，把所有这些单词的产生概率乘起来，数值越大代表这越像一句人话。

<img src="img/nnlm_1.png" width="500" height="450">

现在设计一个神经网络结构来做这个语言模型任务，即给定大量语料来训练一个神经网络，训练好之后，输入一句话的前面几个单词，这个网络可以输出后面紧跟的单词应该是哪个，你会怎么做？

<img src="img/nnlm_2.png" width="500" height="450">

可以像上图这么设计网络结构，这就是NNLM，是Bengio在2003年发表在JMLR上的论文。学习任务是输入某个句中单词 $W_t = "Bert"$ 前面句子的t-1个单词，要求网络正确预测单词Bert，即最大化:$P(W_t = "Bert"|W_1,W_2,...W_{(t-1)};\theta)$。前面任意单词 $W_i$ 用onehot编码（比如：0001000）作为原始单词输入，之后乘以矩阵Q获得向量 $C(W_i)$ ，每个单词的 $C(W_i)$ 进行拼接，上接隐层，然后接softmax去预测后面应该接哪个单词。这个 $C(W_i)$ 是什么？

其实就是单词对应的Word Embedding值，那个矩阵Q包含V行(词典大小)，每一行内容代表对应单词的Word embedding值。只不过Q的内容也是网络参数，需要学习获得，训练刚开始用随机值初始化矩阵Q，当这个网络训练好之后，矩阵Q的内容被正确赋值，每一行代表一个单词对应的Word embedding值。所以你看，通过这个网络学习语言模型任务，这个网络不仅自己能够根据上文预测后接单词是什么，同时获得一个副产品，就是那个矩阵Q，这就是单词的Word Embedding是被如何学会的。

### Word2Vec

Word2Vec的网络结构和NNLM是基本类似的，不过需要指出：尽管网络结构相近，而且也是做语言模型任务，但是其训练方法不太一样。Word2Vec有两种训练方法:
* CBOW: 从一个句子里面把一个词抠掉，用这个词的上文和下文去预测被抠掉的这个词；
* Skip-gram: 和CBOW正好反过来，输入某个单词，要求网络预测它的上下文单词。（实现见Summary3-Skip-gram.ipynb）

<img src="img/word2vec.png" width="500" height="450">

### 训练过程

#### 1. 生成训练样本

##### 使用滑动窗口生成CBOW训练样本 (window_size=3)：

<img src="img/cbow.png" width="500" height="450">

##### 使用滑动窗口生成Skipgram训练样本 (window_size=5)：

<img src="img/skip.png" width="500" height="450">

#### 2. 负采样

神经语言模型计算其预测的三个步骤：

<img src="img/neg_1.png" width="500" height="350">

从计算的角度来看第三步非常消耗资源：尤其是在数据集中为每个训练样本做一次，全部计算下来很可能数千万次。因为我们专注于嵌入。要使用高性能模型生成高质量嵌入，可以从预测相邻单词切换模型的任务为：一个取输入和输出字的模型并输出一个分数，表明它们是否是邻居（0表示“不是邻居”，1表示“邻居”）。

<img src="img/neg_2.png" width="500" height="450">

这个简单的改变，将我们需要的模型从神经网络改为逻辑回归模型：因此它变得更简单，计算速度更快。这个改变要求我们切换数据集的结构 - 标签，现在是一个值为0或1的新列。它们将全部为1，因为我们添加的所有单词都是邻居。

<img src="img/neg_3.png" width="500" height="450">

但这里存在一个漏洞，如果所有的例子都是正面的，模型的可能性总是返回1，达到100%的准确性，但是什么都不学习并生成垃圾嵌入。

<img src="img/neg_4.png" width="400" height="350">

为了解决这个问题，需要在数据集中引入负样本 - 不是邻居的单词样本。我们的模型需要为这些样本返回0。

<img src="img/neg_5.png" width="500" height="450">

#### 3. 预处理

* 确定词汇量的大小（我们称之为vocab_size）；
* 创建两个矩阵 - Embedding矩阵和Context矩阵。这两个矩阵在我们的词汇表中嵌入了每个单词（这vocab_size是他们的维度之一）。第二个维度是我们希望每次嵌入的时间长度（embedding_size- 300是一个常见值）。
* 用随机值初始化这些矩阵。

<img src="img/init.png" width="500" height="450">

#### 4. 训练示例

在每个训练步骤中，采取一个正样本及其相关的负样本。我们来看看我们的第一组：
<img src="img/train1.png" width="500" height="450">

现在有四个单词：输入单词not和输出/上下文单词:( thou实际邻居）,aaron和taco（负样本）。继续查找它们的嵌入 - 对于输入词，我们查看Embedding矩阵。对于上下文单词，我们查看Context矩阵（即使两个矩阵都在我们的词汇表中嵌入了每个单词）。

<img src="img/train2.png" width="500" height="450">

然后，计算输入嵌入与每个上下文嵌入的点积。在每种情况下，会产生一个数字，该数字表示输入和上下文嵌入的相似性。

<img src="img/train3.png" width="500" height="450">

现在需要一种方法将这些分数转化为看起来像概率的东西 ：使用sigmoid函数把概率转换为0和1。
接下来可以将sigmoid操作的输出视为这些样本的模型输出。我们有一个实际的目标标签要比较，那么让我们计算模型预测中的误差。为此，我们只从目标标签中减去sigmoid分数。

<img src="img/train4.png" width="500" height="450">

上面是“机器学习”的“学习”部分。现在，我们可以利用这个错误分数调整`not`，`thou`，`aaron`和`taco`的嵌入，使下一次做出计算结果会更接近目标分数。

<img src="img/train5.png" width="500" height="450">

训练步骤到此结束。我们从这一步骤中得到稍微好一点的嵌入（`not`，`thou`，`aaron`和`taco`）。我们现在进行下一步（下一个正样本及其相关的负样本）,并再次执行相同的过程。

<img src="img/train6.png" width="500" height="450">

当我们循环遍历整个数据集多次时，嵌入继续得到改进。然后我们可以停止训练过程，丢弃`Context`矩阵，并使用`Embeddings`矩阵作为下一个任务的预训练嵌入。

#### 5.超参数的设置

word2vec训练过程中的两个关键超参数是

(1) `窗口大小`

* 较小的窗口嵌入（2-15）:其中两个嵌入之间的高相似性得分表明这些单词是可互换的（注意，如果我们只查看周围的单词，反义词通常可以互换 - 例如，好的和坏的经常出现在类似的情境中）。

* 较大的窗口嵌入（15-50，甚至更多）:会得到相似性更能指示单词相关性的嵌入。实际上，您通常需要对嵌入过程提供注释指导，为您的任务带来有用的相似感。

Gensim默认窗口大小为5（输入字本身加上输入字之前的两个字和输入字之后的两个字）。

(2) `负样本数量`

负样本的数量是训练过程的另一个因素。原始论文里负样本数量为5-20。它还指出，当你拥有足够大的数据集时，2-5似乎已经足够了。Gensim默认为5个负样本。

<img src="img/word2vec_neg.png" width="500" height="450">

### GloVe

虽然 Word2Vec 已经能够成功地将离散的单词转换为连续的词向量，并能一定程度上地保存词与词之间的近似关系，但 Word2Vec 模型仍不是完美的，它还可以被进一步地改进：

1. 子词嵌入（subword embedding）：[FastText](https://zh.d2l.ai/chapter_natural-language-processing/fasttext.html) 以固定大小的 n-gram 形式将单词更细致地表示为子词的集合，而 [BPE (byte pair encoding)](https://d2l.ai/chapter_natural-language-processing/subword-embedding.html#byte-pair-encoding) 算法则能根据语料库的统计信息，自动且动态地生成高频子词的集合；
2. [GloVe 全局向量的词嵌入](https://zh.d2l.ai/chapter_natural-language-processing/glove.html): 通过等价转换 Word2Vec 模型的条件概率公式，我们可以得到一个全局的损失函数表达，并在此基础上进一步优化模型。

Word2Vec的损失函数（以 Skip-Gram 模型为例，不考虑负采样近似）：$-\sum_{t=1}^T\sum_{-m\le j\le m,j\ne 0} \log P(w^{(t+j)}\mid w^{(t)})$

在Word2Vec基础上做出了以下几点改动：

1. 使用非概率分布的变量 $p'_{ij}=x_{ij}$ 和 $q′_{ij}=\exp(\boldsymbol{u}^\top_j\boldsymbol{v}_i)$，并对它们取对数；
2. 为每个词 $w_i$ 增加两个标量模型参数：中心词偏差项 $b_i$ 和背景词偏差项 $c_i$，松弛了概率定义中的规范性；
3. 将每个损失项的权重 $x_i$ 替换成函数 $h(x_{ij})$，权重函数 $h(x)$ 是值域在 $[0,1]$ 上的单调递增函数，松弛了中心词重要性与 $x_i$ 线性相关的隐含假设；
4. 用平方损失函数替代了交叉熵损失函数。

获得了Glove的损失函数：

$\sum_{i\in\mathcal{V}}\sum_{j\in\mathcal{V}} h(x_{ij}) (\boldsymbol{u}^\top_j\boldsymbol{v}_i+b_i+c_j-\log x_{ij})^2$

由于这些非零 $x_{ij}$ 是预先基于整个数据集计算得到的，包含了数据集的全局统计信息，因此 GloVe 模型的命名取“全局向量”（Global Vectors）之意。

**载入预训练的GloVe向量**

[GloVe 官方](https://nlp.stanford.edu/projects/glove/) 提供了多种规格的预训练词向量，语料库分别采用了维基百科、CommonCrawl和推特等，语料库中词语总数也涵盖了从60亿到8,400亿的不同规模，同时还提供了多种词向量维度供下游模型使用。

[`torchtext.vocab`](https://torchtext.readthedocs.io/en/latest/vocab.html) 中已经支持了 GloVe, FastText, CharNGram 等常用的预训练词向量，我们可以通过声明 [`torchtext.vocab.GloVe`](https://torchtext.readthedocs.io/en/latest/vocab.html#glove) 类的实例来加载预训练好的 GloVe 词向量。

In [3]:
import torch
import torchtext.vocab as vocab

print([key for key in vocab.pretrained_aliases.keys() if "glove" in key])
cache_dir = "D:\IIE\pytorch\jupyter_model\data"
glove = vocab.GloVe(name='6B', dim=50, cache=cache_dir)
print("一共包含%d个词。" % len(glove.stoi))
print(glove.stoi['beautiful'], glove.itos[3366])

['glove.42B.300d', 'glove.840B.300d', 'glove.twitter.27B.25d', 'glove.twitter.27B.50d', 'glove.twitter.27B.100d', 'glove.twitter.27B.200d', 'glove.6B.50d', 'glove.6B.100d', 'glove.6B.200d', 'glove.6B.300d']
一共包含400000个词。
3366 beautiful


100%|██████████████████████████████████████████████████████████████████████▉| 399953/400000 [00:40<00:00, 16015.05it/s]